In [ ]:
library(ggplot2)
library(reshape2)
library(repr)
library(plyr)
library(dplyr)
library(ggthemes)

options(repr.plot.width=4, repr.plot.height=3)

In [ ]:
xs <- read.csv('solver-1502985187.csv', h=T)
xs$erasure = mapvalues(
    xs$erasure,
    from = c("True", "False"),
    to   = c(1, 0)
)
str(xs)

In [ ]:
options(repr.plot.width=4, repr.plot.height=3)
ggplot(xs, aes(annotations, duration, colour=erasure)) +
    geom_point()

In [ ]:
xs %>%
    filter(annotations < 200 & duration > 0.025) %>%
    arrange(desc(duration))

In [ ]:
xs_averaged <- xs %>%
    group_by(program, annotations, annotations_used, erasure) %>%
    summarise(
        duration_mean = mean(duration),
        duration_sd = sd(duration)
    )

In [ ]:
diff <- function(xs) {
    if (length(xs) == 2) {
        xs[2] - xs[1]
    } else {
        NA
    }
}

xs_diff <- xs_averaged %>%
    group_by(program, annotations) %>%
    summarise(
        annotations_used = min(annotations_used),
        difference = diff(duration_mean),
        difference_sd = sum(duration_sd)
    )
xs_diff

In [ ]:
mytheme <- theme_minimal() +
    theme(
        axis.text=element_text(size=8),
        axis.title=element_text(size=8)
    )

options(repr.plot.width=3, repr.plot.height=3)
ggplot(xs_diff %>% filter(difference < 0), aes(annotations, -1000 * difference)) +
    geom_point(size=0.5) +
    geom_text(aes(label=program), size=3, vjust=0.3, hjust=1, nudge_x=-15) +
    #ylim(0, 40) +
    ylab("inference time [ms]") +
    mytheme

In [ ]:
ggplot(xs_diff %>% filter(difference < 0), aes(annotations_used, -1000 * difference)) +
    geom_point(size=0.5) +
    geom_text(aes(label=program), size=3, vjust=0.3, hjust=1, nudge_x=-15) +
    #ylim(0, 40) +
    ylab("inference time [ms]") +
    mytheme